In [7]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='X92ykei0YBdvsxzGx5ZR4F8F5m-oqRAnov2j9kfoxIv7',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'aipowerednutritionanalyzerforfitn-donotdelete-pr-rwkepbgj3qtrfx'
object_key = 'TRAIN_SET.zip'

streaming_body_1 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


In [8]:
from io import BytesIO
import zipfile
unzip=zipfile.ZipFile(BytesIO(streaming_body_1.read()),'r')
file_paths=unzip.namelist()
for path in file_paths:
    unzip.extract(path)

In [9]:
ls

'nutrition(2).h5'   TEST_SET-20221117T092410Z-001.zip   TRAIN_SET/


In [10]:
pwd

'/home/wsuser/work'

In [11]:
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import tensorflow 
from tensorflow.keras.models import Sequential 
from tensorflow.keras import layers 
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D,Dropout
from keras.preprocessing.image import ImageDataGenerator

In [12]:
train_datagen = ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
test_datagen=ImageDataGenerator(rescale=1./255)

In [13]:
x_train = train_datagen.flow_from_directory(
    r'/home/wsuser/work/TRAIN_SET',
    target_size=(64, 64),batch_size=5,color_mode='rgb',class_mode='sparse')
x_test = test_datagen.flow_from_directory(
    r'/home/wsuser/work/TRAIN_SET',
    target_size=(64, 64),batch_size=5,color_mode='rgb',class_mode='sparse')

Found 2626 images belonging to 5 classes.
Found 2626 images belonging to 5 classes.


In [14]:
print(x_train.class_indices)

{'APPLES': 0, 'BANANA': 1, 'ORANGE': 2, 'PINEAPPLE': 3, 'WATERMELON': 4}


In [15]:
print(x_test.class_indices)

{'APPLES': 0, 'BANANA': 1, 'ORANGE': 2, 'PINEAPPLE': 3, 'WATERMELON': 4}


In [16]:
from collections import Counter as c 
c(x_train .labels)

Counter({0: 606, 1: 445, 2: 479, 3: 621, 4: 475})

In [17]:
model=Sequential()

In [18]:
classifier = Sequential()
classifier.add(Conv2D(32,(3, 3), input_shape=(64, 64, 3),activation='relu')) 
classifier.add(MaxPooling2D(pool_size=(2, 2)))
classifier.add(Conv2D(32, (3, 3), activation='relu')) 
classifier.add(MaxPooling2D(pool_size=(2, 2)))
classifier.add(Flatten())
classifier.add(Dense (units=128, activation='relu'))
classifier.add(Dense (units=5, activation='softmax'))

In [19]:
classifier.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 31, 31, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 14, 14, 32)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 6272)              0         
                                                                 
 dense (Dense)               (None, 128)              

In [20]:
classifier.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [21]:
classifier.fit_generator(
    generator=x_train,steps_per_epoch = len(x_train), 
    epochs=20,validation_data=x_test,validation_steps = len(x_test)) 

/tmp/wsuser/ipykernel_164/2585906849.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  classifier.fit_generator(


Epoch 1/20
526/526 [==============================] - 15s 27ms/step - loss: 0.1704 - accuracy: 0.9330 - val_loss: 0.0013 - val_accuracy: 1.0000
Epoch 2/20
526/526 [==============================] - 14s 26ms/step - loss: 0.0030 - accuracy: 0.9989 - val_loss: 0.0021 - val_accuracy: 1.0000
Epoch 3/20
526/526 [==============================] - 14s 26ms/step - loss: 3.8164e-04 - accuracy: 1.0000 - val_loss: 3.4988e-05 - val_accuracy: 1.0000
Epoch 4/20
526/526 [==============================] - 14s 26ms/step - loss: 0.0288 - accuracy: 0.9916 - val_loss: 7.8114e-04 - val_accuracy: 1.0000
Epoch 5/20
526/526 [==============================] - 14s 26ms/step - loss: 0.0183 - accuracy: 0.9928 - val_loss: 3.2802e-04 - val_accuracy: 1.0000
Epoch 6/20
526/526 [==============================] - 14s 26ms/step - loss: 0.0038 - accuracy: 0.9981 - val_loss: 5.1809e-05 - val_accuracy: 1.0000
Epoch 7/20
526/526 [==============================] - 14s 26ms/step - loss: 1.0688e-04 - accuracy: 1.0000 - val_loss

In [22]:
classifier.save('nutrition(2).h5')

In [23]:
from tensorflow.keras.models import load_model
from keras.preprocessing import image
model = load_model("nutrition(2).h5")
from tensorflow.keras.preprocessing import image

In [24]:
img =image.load_img(r"/home/wsuser/work/TRAIN_SET/ORANGE/100_100.jpg",grayscale=False, target_size= (64,64))
x = image.img_to_array(img)
x = np.expand_dims(x,axis=0) 
y =np.argmax(model.predict(x),axis=1)
index=['APPLES', 'BANANA', 'ORANGE','PINEAPPLE','WATERMELON']
index[y[0]]

'ORANGE'

 IBM Deployment

In [25]:
!pip install ibm_watson_machine_learning

In [26]:
!pip install keras==2.2.4
!pip install tensorflow==2.5.0

     |████████████████████████████████| 312 kB 21.3 MB/s eta 0:00:01
     |████████████████████████████████| 50 kB 14.9 MB/s eta 0:00:01
  Attempting uninstall: keras
    Found existing installation: keras 2.7.0
    Uninstalling keras-2.7.0:
      Successfully uninstalled keras-2.7.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.7.2 requires keras<2.8,>=2.7.0, but you have keras 2.2.4 which is incompatible.
     |████████████████████████████████| 454.4 MB 38 kB/s s eta 0:00:01
     |████████████████████████████████| 4.0 MB 57.7 MB/s eta 0:00:01
     |████████████████████████████████| 14.9 MB 64.7 MB/s eta 0:00:01
     |████████████████████████████████| 4.4 MB 64.5 MB/s eta 0:00:01
     |████████████████████████████████| 462 kB 65.7 MB/s eta 0:00:01
     |████████████████████████████████| 1.2 MB 65.2 MB/s eta 0:00:01
  Attempting uninstall: numpy


      Successfully uninstalled grpcio-1.42.0
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.1.1
    Uninstalling typing-extensions-4.1.1:
      Successfully uninstalled typing-extensions-4.1.1
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.7.0
    Uninstalling tensorflow-estimator-2.7.0:
      Successfully uninstalled tensorflow-estimator-2.7.0
  Attempting uninstall: h5py
    Found existing installation: h5py 3.2.1
    Uninstalling h5py-3.2.1:
      Successfully uninstalled h5py-3.2.1
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 2.0
    Uninstalling flatbuffers-2.0:
      Successfully uninstalled flatbuffers-2.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.7.2
    Uninstalling tensorflow-2.7.2:
      Successfully uninstalled tensorflow-2.7.2
ERROR: pip's dependency resolver does not currently take into account all

In [29]:
from ibm_watson_machine_learning import APIClient

In [30]:
wml_credentials={
    "url":"https://us-south.ml.cloud.ibm.com",
    "apikey":"VH626RcYUA2hhHu3Fazm3Afn9OlBFKFbj2q0GNB6jZMo"
}

In [32]:
client=APIClient(wml_credentials)

In [33]:
client

In [34]:
client.spaces.list()

Note: 'limit' is not provided. Only first 50 records will be displayed if the number of records exceed 50
------------------------------------  ---------  ------------------------
ID                                    NAME       CREATED
d5b1927a-cf18-4a1c-be40-4ace2d0984c1  nutrition  2022-11-17T10:29:41.811Z
------------------------------------  ---------  ------------------------


In [35]:
space_id="d5b1927a-cf18-4a1c-be40-4ace2d0984c1"

In [36]:
client.set.default_space(space_id)

'SUCCESS'

In [37]:
client.software_specifications.list()

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12      020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9      0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod   1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl      111e41b3-de2d-5422-a4d6-bf776828c4b7  base
autoai-kb_rt22.2-py3.10        125b6d9

In [38]:
software_space_id = client.software_specifications.get_id_by_name("tensorflow_rt22.1-py3.9")

In [39]:
software_space_id

'acd9c798-6974-5d2f-a657-ce06e986df4d'

In [54]:
!tar -zcvf nutrition-classification-model.tgz 'nutrition(2).h5'

nutrition(2).h5


In [56]:
model_details=client.repository.store_model(model ='nutrition-classification-model.tgz', meta_props={
    client.repository.ModelMetaNames.NAME: "CNN Model Building",
    client.repository.ModelMetaNames.TYPE: 'tensorflow_2.7',
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: software_space_id
})

In [57]:
model_id = client.repository.get_model_id(model_details)

In [58]:
model_id

'c5f3a7e0-8b23-4a85-b59b-498f77e1622c'